<a href="https://colab.research.google.com/github/mauriciodev/braz_iono_series/blob/main/example_data_usage.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

MIT License

Copyright (c) 2022 Mauricio Carvalho Mathias de Paulo and Danilo Souza

Permission is hereby granted, free of charge, to any person obtaining a copy of this software and associated documentation files (the "Software"), to deal in the Software without restriction, including without limitation the rights to use, copy, modify, merge, publish, distribute, sublicense, and/or sell copies of the Software, and to permit persons to whom the Software is furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.

In [ ]:
from datetime import datetime
import pandas as pd
import urllib
import os
from zipfile import ZipFile
import subprocess
import numpy as np
import matplotlib.pyplot as plt
import re
import io
import urllib.request
import scipy
from sklearn.metrics import r2_score
from sklearn.linear_model import LinearRegression

# Baixando o arquivo do github

In [ ]:
url="https://raw.githubusercontent.com/mauriciodev/braz_iono_series/main/iono_series.csv"
!wget --no-check-certificate $url

In [ ]:
df=pd.read_csv("iono_series.csv")
df['date']=pd.to_datetime(df['date'])#.astype('datetime64[D]') #recognizes date column as date type
df

# Limpando anomalias após o processamento

In [ ]:
#|X-Xavg|<3*std
#column='max_VTEC'
column='maxA_I2L1'
nstd=3 #number of standard deviations above or bellow median
plt.figure(figsize=(15, 5))
s=df[column].rolling(9, center=True, min_periods=3).std()
median=df[column].rolling(9, center=True, min_periods=3).median(skipna=True)
plt.plot(median, label="Median")
#outliers=abs(df[column]-median)>nstd*s
outliers=abs(df[column]-median)>0.003
plt.plot(df[column], label=column, marker='.', linestyle="None")
plt.plot(df[outliers][column],marker='o',linestyle = 'None', label="Outliers")
plt.legend()

In [ ]:
df[outliers]

# Removendo da série

In [ ]:
df[outliers] = np.nan

# Estatística descritiva

In [ ]:
cols=["mean_I2L1","mean_I3L1","mean_I2L2","mean_I3L2", "mean_VTEC", "maxA_I2L1","maxA_I3L1","maxA_I2L2","maxA_I3L2" ]
df[cols].describe()

# Gráficos individuais

In [ ]:
plt.plot(df['date'],df['f107cm'], label='F10.7cm (fluxo solar)')
plt.xlabel('Ano')
plt.ylabel('F10.7cm')
plt.legend()

In [ ]:
plt.figure(figsize=(20,20))
plt.plot(df[['mean_I3L2']], label="L2 - 3a ordem")
plt.plot(df[['mean_I3L1']], label="L1 - 3a ordem")
plt.plot(df[['mean_I2L2']], label="L2 - 2a ordem")
plt.plot(df[['mean_I2L1']], label="L1 - 2a ordem")
plt.plot(df[['f107cm']]/100000., label="F10.7cm")
plt.title('Atrasos ionosféricos de 2ª ordem')
plt.xlabel('Dia do ano')
plt.ylabel('Atraso ionosférico de 2ª ordem (m)')
plt.legend()
plt.show()

# Correlação

In [ ]:
#Correlation using pandas
#df2=df.ffill()
df[cols+["f107cm"]].corr(method='pearson')

In [ ]:
df2=df.dropna() #removing nan's
compareSeries=df2[['mean_I2L1']]
plt.scatter(df2[['f107cm']],compareSeries)
plt.annotate("r-squared = {:.3f}".format(r2_score(df2[['f107cm']], compareSeries)), (0, 1), xycoords='figure points')
y_test, y_predicted = df2[['f107cm']].to_numpy().reshape(-1,1), np.array(compareSeries).reshape(-1,1)
plt.plot(y_test, LinearRegression().fit(y_test, y_predicted).predict(y_test), color="red")

# Gráficos das séries

In [ ]:
fig, axs = plt.subplots(4, figsize=(15, 10))
fig.tight_layout()
axs[0].set_title('Média diária de atrasos ionosféricos de ordem superior (mm)')
axs[0].plot(df['date'],df['mean_I3L2']*1000, label="L2 - 3a ordem")
axs[0].plot(df['date'],df['mean_I3L1']*1000, label="L1 - 3a ordem")
axs[0].plot(df['date'],df['mean_I2L2']*1000, label="L2 - 2a ordem")
axs[0].plot(df['date'],df['mean_I2L1']*1000, label="L1 - 2a ordem")
axs[0].legend()
axs[1].set_title('Média diária do VTEC (TECU)')
axs[1].plot(df['date'],df['mean_VTEC'], label="VTEC")
axs[1].legend()
axs[2].set_title('F10.7cm (índice de fluxo solar)')
axs[2].plot(df['date'],df['f107cm'], label="F10.7cm")
axs[2].legend()
axs[3].set_title('Ap (índice de atividade geomagnética)')
axs[3].plot(df['date'],df['ap'], label="Ap")
axs[3].legend()
plt.xlabel('Ano')
plt.show()

In [ ]:

fig, axs = plt.subplots(6, figsize=(15, 10))
fig.tight_layout()
axs[0].set_title('Média diária de atrasos ionosféricos de ordem superior (mm)')
axs[0].plot(df['date'],df['mean_I3L2']*1000, label="L2 - 3a ordem")
axs[0].plot(df['date'],df['mean_I3L1']*1000, label="L1 - 3a ordem")
axs[0].plot(df['date'],df['mean_I2L2']*1000, label="L2 - 2a ordem")
axs[0].plot(df['date'],df['mean_I2L1']*1000, label="L1 - 2a ordem")
axs[0].legend()
axs[1].set_title('Máximos diários dos atrasos ionosféricos de ordem superior (mm)')
axs[1].plot(df['date'],df['max_I3L2']*1000, label="L2 - 3a ordem")
axs[1].plot(df['date'],df['max_I3L1']*1000, label="L1 - 3a ordem")
axs[1].plot(df['date'],df['max_I2L2']*1000, label="L2 - 2a ordem")
axs[1].plot(df['date'],df['max_I2L1']*1000, label="L1 - 2a ordem")
axs[1].legend()
axs[2].set_title('Mínimo diário dos atrasos ionosféricos de ordem superior (mm)')
axs[2].plot(df['date'],df['min_I3L2']*1000, label="L2 - 3a ordem")
axs[2].plot(df['date'],df['min_I3L1']*1000, label="L1 - 3a ordem")
axs[2].plot(df['date'],df['min_I2L2']*1000, label="L2 - 2a ordem")
axs[2].plot(df['date'],df['min_I2L1']*1000, label="L1 - 2a ordem")
axs[2].legend()
axs[3].set_title('Média diária do VTEC (TECU)')
axs[3].plot(df['date'],df['mean_VTEC'], label="VTEC")
axs[3].legend()
axs[4].set_title('F10.7cm (índice de fluxo solar)')
axs[4].plot(df['date'],df['f107cm'], label="F10.7cm")
axs[4].legend()
axs[5].set_title('Ap (índice de atividade geomagnética)')
axs[5].plot(df['date'],df['ap'], label="Ap")
axs[5].legend()
plt.xlabel('Ano')
plt.show()


## Gráfico com valores absolutos


In [ ]:
fig, axs = plt.subplots(5, figsize=(15, 10))
fig.tight_layout()
i=0
axs[i].set_title('Média diária dos valores dos atrasos ionosféricos de ordem superior (mm)')
axs[i].plot(df['date'],df['mean_I3L2']*1000, label="L2 - 3a ordem")
axs[i].plot(df['date'],df['mean_I3L1']*1000, label="L1 - 3a ordem")
axs[i].plot(df['date'],df['mean_I2L2']*1000, label="L2 - 2a ordem")
axs[i].plot(df['date'],df['mean_I2L1']*1000, label="L1 - 2a ordem")
axs[i].legend(loc=1)
i+=1
axs[i].set_title('Média diária dos valores absolutos dos atrasos ionosféricos de ordem superior (mm)')
axs[i].plot(df['date'],df['meanA_I3L2']*1000, label="L2 - 3a ordem")
axs[i].plot(df['date'],df['meanA_I3L1']*1000, label="L1 - 3a ordem")
axs[i].plot(df['date'],df['meanA_I2L2']*1000, label="L2 - 2a ordem")
axs[i].plot(df['date'],df['meanA_I2L1']*1000, label="L1 - 2a ordem")
axs[i].legend(loc=1)
i+=1
axs[i].set_title('Máximos diários dos valores absolutos dos atrasos ionosféricos de ordem superior (mm)')
axs[i].plot(df['date'],df['maxA_I3L2']*1000, label="L2 - 3a ordem")
axs[i].plot(df['date'],df['maxA_I3L1']*1000, label="L1 - 3a ordem")
axs[i].plot(df['date'],df['maxA_I2L2']*1000, label="L2 - 2a ordem")
axs[i].plot(df['date'],df['maxA_I2L1']*1000, label="L1 - 2a ordem")
axs[i].legend(loc=1)
i+=1
"""axs[i].set_title('Mínimos diários dos valores absolutos dos atrasos ionosféricos de ordem superior (mm)')
axs[i].plot(df['date'],df['minA_I3L2']*1000, label="L2 - 3a ordem")
axs[i].plot(df['date'],df['minA_I3L1']*1000, label="L1 - 3a ordem")
axs[i].plot(df['date'],df['minA_I2L2']*1000, label="L2 - 2a ordem")
axs[i].plot(df['date'],df['minA_I2L1']*1000, label="L1 - 2a ordem")
axs[i].legend(loc=1)
i+=1"""
axs[i].set_title('Média diária do VTEC (TECU)')
axs[i].plot(df['date'],df['mean_VTEC'], label="VTEC")
axs[i].legend(loc=1)
i+=1
axs[i].set_title('F10.7cm (índice de fluxo solar)')
axs[i].plot(df['date'],df['f107cm'], label="F10.7cm")
axs[i].legend(loc=1)
"""i+=1
axs[i].set_title('Ap (índice de atividade geomagnética)')
axs[i].plot(df['date'],df['ap'], label="Ap")
axs[i].legend(loc=1)"""
plt.xlabel('Ano')
plt.savefig("absValues.svg")
plt.show()